In [21]:
import pandas as pd
from ics import Calendar, Event, DisplayAlarm
from datetime import datetime, timedelta

# read scraped output
df = pd.read_csv("output.csv", names=["artist","url","artist_country","location","date_string","insta","spotify","rf_short_description"])
df = df.set_index('artist')

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment','emoji-beskrivelse 🎨'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment','emoji-beskrivelse 🎨':'emoji_beskrivelse'})
df_ratings = df_ratings.set_index('artist')
df_ratings[['comment', 'm_rating','l_rating','emoji_beskrivelse']] = df_ratings[['comment', 'm_rating','l_rating','emoji_beskrivelse']].fillna(' ')



In [22]:
# join website info with ratings  
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')


In [23]:
# get starttime, fill nulls with timestamp 13:37
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})').fillna('13:37')


# strip irrelevant locale information; could probably be made redundant
df['new_date_string'] = df['date_string'].replace(to_replace=r'.*dag (\d{1,2})\. (\w{3})\w', value=r'\1\2', regex=True)

# convert to timestamp w timezone; ics package needs utc input
df['start_datetime'] = pd.to_datetime(df['new_date_string'] + df['start_time'], format='%d%b %Y%H:%M').dt.tz_localize('CET')

## todo handle time of day for ical. Example below
df[['date_string','start_datetime','start_time']][125:].head()

date_string            start_datetime start_time
artist                                                                       
Kesi                Fredag 30. juni 2023 2023-06-30 13:37:00+02:00      13:37
Kind Mod Kind       Søndag 25. juni 2023 2023-06-25 13:37:00+02:00      13:37
Kindsight           Mandag 26. juni 2023 2023-06-26 13:37:00+02:00      13:37
King Kami      21:45 Lørdag 1. juli 2023 2023-07-01 21:45:00+02:00      21:45
Kinga Bartis        Onsdag 28. juni 2023 2023-06-28 13:37:00+02:00      13:37

In [24]:
#df['location'].unique()
stages = {
    'Orange': '🟠 Orange',
    'Arena': '🎪 Arena',
    'Avalon': '🏝️ Avalon',
    'TBA': '❔ TBA',
    'Platform': 'Platform',
    'Gloria': '🏝️ Gloria',
    'Apollo': '🚀 Apollo',
    'Festival Site': '🏟️ Festival Site',
    'Campsite': '⛺ Campsite',
    'Destroy Me Once, Destroy Me Twice': '☠️ Destroy Me Once, Destroy Me Twice',
    'Flokkr': '📣 Flokkr',
    'Food Is Now': '🥫 Food Is Now'}

df['location'] = df['location'].map(stages)

In [25]:
# write most important concerts to ical
c = Calendar()
for index, row in df[(df['m_rating'] == '🤯JA!') | (df['m_rating'] == '👍Ja' )].iterrows():
    e = Event()

    abbreviated_comment = ' | ' + row['comment'] if len(row['comment']) > 1 else ''
    line_rating = ' | L: ' + row['l_rating'][0] if len(row['l_rating']) > 1 else ''

    # add name and location + short info 
    e.name = row['m_rating'][0] + " " + index + line_rating
    e.location = row['location'] + abbreviated_comment

    e.begin = row['start_datetime']

    # assume all concerts are 1 hour
    e.end = row['start_datetime'] + timedelta(hours=1) 

    # add url and description
    e.url = row['url']
    e.description = '"' + row['rf_short_description'] + '"\n▶️ ' + row['spotify'] + '\n 📸 ' + row['insta']

    # only add alarm for the most important concerts
    if row['m_rating'] == '🤯JA!':
        a=DisplayAlarm()
        a.trigger=timedelta(hours=-1)
        e.alarms=[a]
    
    c.events.add(e)

# c.events
with open('most_important.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())


/Users/mikaelthorup/repos/RF23/venv/lib/python3.10/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


In [26]:
# thx for the random_emoji_string-function, chatGPT <3

import random

emojis = [ # from https://getemoji.com/
    '😀','😃','😄','😁','😆','😅','😂','🤣','🥲','🥹','☺️','😊','😇','🙂','🙃','😉','😌','😍','🥰','😘','😗','😙','😚','😋','😛','😝','😜','🤪','🤨','🧐','🤓','😎','🥸','🤩','🥳','😏','😒','😞','😔','😟','😕','🙁','☹️','😣','😖','😫','😩','🥺','😢','😭','😮‍💨','😤','😠','😡','🤬','🤯','😳','🥵','🥶','😱','😨','😰','😥','😓','🫣','🤗','🫡','🤔','🫢','🤭','🤫','🤥','😶','😶‍🌫️','😐','😑','😬','🫨','🫠','🙄','😯','😦','😧','😮','😲','🥱','😴','🤤','😪','😵','😵‍💫','🫥','🤐','🥴','🤢','🤮','🤧','😷','🤒','🤕','🤑','🤠','😈','👿','👹','👺','🤡','💩','👻','💀','☠️','👽','👾','🤖','🎃','😺','😸','😹','😻','😼','😽','🙀','😿','😾'
    ,'👋','🤚','🖐','✋','🖖','👌','🤌','🤏','✌️','🤞','🫰','🤟','🤘','🤙','🫵','🫱','🫲','🫸','🫷','🫳','🫴','👈','👉','👆','🖕','👇','☝️','👍','👎','✊','👊','🤛','🤜','👏','🫶','🙌','👐','🤲','🤝','🙏','✍️','💅','🤳','💪','🦾','🦵','🦿','🦶','👣','👂','🦻','👃','🫀','🫁','🧠','🦷','🦴','👀','👁','👅','👄','🫦','💋','🩸'
    ,'👶','👧','🧒','👦','👩','🧑','👨','👩‍🦱','🧑‍🦱','👨‍🦱','👩‍🦰','🧑‍🦰','👨‍🦰','👱‍♀️','👱','👱‍♂️','👩‍🦳','🧑‍🦳','👨‍🦳','👩‍🦲','🧑‍🦲','👨‍🦲','🧔‍♀️','🧔','🧔‍♂️','👵','🧓','👴','👲','👳‍♀️','👳','👳‍♂️','🧕','👮‍♀️','👮','👮‍♂️','👷‍♀️','👷','👷‍♂️','💂‍♀️','💂','💂‍♂️','🕵️‍♀️','🕵️','🕵️‍♂️','👩‍⚕️','🧑‍⚕️','👨‍⚕️','👩‍🌾','🧑‍🌾','👨‍🌾','👩‍🍳','🧑‍🍳','👨‍🍳','👩‍🎓','🧑‍🎓','👨‍🎓','👩‍🎤','🧑‍🎤','👨‍🎤','👩‍🏫','🧑‍🏫','👨‍🏫','👩‍🏭','🧑‍🏭','👨‍🏭','👩‍💻','🧑‍💻','👨‍💻','👩‍💼','🧑‍💼','👨‍💼','👩‍🔧','🧑‍🔧','👨‍🔧','👩‍🔬','🧑‍🔬','👨‍🔬','👩‍🎨','🧑‍🎨','👨‍🎨','👩‍🚒','🧑‍🚒','👨‍🚒','👩‍✈️','🧑‍✈️','👨‍✈️','👩‍🚀','🧑‍🚀','👨‍🚀','👩‍⚖️','🧑‍⚖️','👨‍⚖️','👰‍♀️','👰','👰‍♂️','🤵‍♀️','🤵','🤵‍♂️','👸','🫅','🤴','🥷','🦸‍♀️','🦸','🦸‍♂️','🦹‍♀️','🦹','🦹‍♂️','🤶','🧑‍🎄','🎅','🧙‍♀️','🧙','🧙‍♂️','🧝‍♀️','🧝','🧝‍♂️','🧛‍♀️','🧛','🧛‍♂️','🧟‍♀️','🧟','🧟‍♂️','🧞‍♀️','🧞','🧞‍♂️','🧜‍♀️','🧜','🧜‍♂️','🧚‍♀️','🧚','🧚‍♂️','🧌','👼','🤰','🫄','🫃','🤱','👩‍🍼','🧑‍🍼','👨‍🍼','🙇‍♀️','🙇','🙇‍♂️','💁‍♀️','💁','💁‍♂️','🙅‍♀️','🙅','🙅‍♂️','🙆‍♀️','🙆','🙆‍♂️','🙋‍♀️','🙋','🙋‍♂️','🧏‍♀️','🧏','🧏‍♂️','🤦‍♀️','🤦','🤦‍♂️','🤷‍♀️','🤷','🤷‍♂️','🙎‍♀️','🙎','🙎‍♂️','🙍‍♀️','🙍','🙍‍♂️','💇‍♀️','💇','💇‍♂️','💆‍♀️','💆','💆‍♂️','🧖‍♀️','🧖','🧖‍♂️','💅','🤳','💃','🕺','👯‍♀️','👯','👯‍♂️','🕴','👩‍🦽','🧑‍🦽','👨‍🦽','👩‍🦼','🧑‍🦼','👨‍🦼','🚶‍♀️','🚶','🚶‍♂️','👩‍🦯','🧑‍🦯','👨‍🦯','🧎‍♀️','🧎','🧎‍♂️','🏃‍♀️','🏃','🏃‍♂️','🧍‍♀️','🧍','🧍‍♂️','👭','🧑‍🤝‍🧑','👬','👫','👩‍❤️‍👩','💑','👨‍❤️‍👨','👩‍❤️‍👨','👩‍❤️‍💋‍👩','💏','👨‍❤️‍💋‍👨','👩‍❤️‍💋‍👨','👪','👨‍👩‍👦','👨‍👩‍👧','👨‍👩‍👧‍👦','👨‍👩‍👦‍👦','👨‍👩‍👧‍👧','👨‍👨‍👦','👨‍👨‍👧','👨‍👨‍👧‍👦','👨‍👨‍👦‍👦','👨‍👨‍👧‍👧','👩‍👩‍👦','👩‍👩‍👧','👩‍👩‍👧‍👦','👩‍👩‍👦‍👦','👩‍👩‍👧‍👧','👨‍👦','👨‍👦‍👦','👨‍👧','👨‍👧‍👦','👨‍👧‍👧','👩‍👦','👩‍👦‍👦','👩‍👧','👩‍👧‍👦','👩‍👧‍👧','🗣','👤','👥','🫂'
    ,'🧳','🌂','☂️','🧵','🪡','🪢','🪭','🧶','👓','🕶','🥽','🥼','🦺','👔','👕','👖','🧣','🧤','🧥','🧦','👗','👘','🥻','🩴','🩱','🩲','🩳','👙','👚','👛','👜','👝','🎒','👞','👟','🥾','🥿','👠','👡','🩰','👢','👑','👒','🎩','🎓','🧢','⛑','🪖','💄','💍','💼'
    ,'🐶','🐱','🐭','🐹','🐰','🦊','🐻','🐼','🐻‍❄️','🐨','🐯','🦁','🐮','🐷','🐽','🐸','🐵','🙈','🙉','🙊','🐒','🐔','🐧','🐦','🐦‍⬛','🐤','🐣','🐥','🦆','🦅','🦉','🦇','🐺','🐗','🐴','🦄','🐝','🪱','🐛','🦋','🐌','🐞','🐜','🪰','🪲','🪳','🦟','🦗','🕷','🕸','🦂','🐢','🐍','🦎','🦖','🦕','🐙','🦑','🦐','🦞','🦀','🪼','🪸','🐡','🐠','🐟','🐬','🐳','🐋','🦈','🐊','🐅','🐆','🦓','🫏','🦍','🦧','🦣','🐘','🦛','🦏','🐪','🐫','🦒','🦘','🦬','🐃','🐂','🐄','🐎','🐖','🐏','🐑','🦙','🐐','🦌','🫎','🐕','🐩','🦮','🐕‍🦺','🐈','🐈‍⬛','🪽','🪶','🐓','🦃','🦤','🦚','🦜','🦢','🪿','🦩','🕊','🐇','🦝','🦨','🦡','🦫','🦦','🦥','🐁','🐀','🐿','🦔','🐾','🐉','🐲','🌵','🎄','🌲','🌳','🌴','🪹','🪺','🪵','🌱','🌿','☘️','🍀','🎍','🪴','🎋','🍃','🍂','🍁','🍄','🐚','🪨','🌾','💐','🌷','🪷','🌹','🥀','🌺','🌸','🪻','🌼','🌻','🌞','🌝','🌛','🌜','🌚','🌕','🌖','🌗','🌘','🌑','🌒','🌓','🌔','🌙','🌎','🌍','🌏','🪐','💫','⭐️','🌟','✨','⚡️','☄️','💥','🔥','🌪','🌈','☀️','🌤','⛅️','🌥','☁️','🌦','🌧','⛈','🌩','🌨','❄️','☃️','⛄️','🌬','💨','💧','💦','🫧','☔️','☂️','🌊','🌫'
    ,'🍏','🍎','🍐','🍊','🍋','🍌','🍉','🍇','🍓','🫐','🍈','🍒','🍑','🥭','🍍','🥥','🥝','🍅','🍆','🥑','🥦','🫛','🥬','🥒','🌶','🫑','🌽','🥕','🫒','🧄','🧅','🫚','🥔','🍠','🫘','🥐','🥯','🍞','🥖','🥨','🧀','🥚','🍳','🧈','🥞','🧇','🥓','🥩','🍗','🍖','🦴','🌭','🍔','🍟','🍕','🫓','🥪','🥙','🧆','🌮','🌯','🫔','🥗','🥘','🫕','🥫','🍝','🍜','🍲','🍛','🍣','🍱','🥟','🦪','🍤','🍙','🍚','🍘','🍥','🥠','🥮','🍢','🍡','🍧','🍨','🍦','🥧','🧁','🍰','🎂','🍮','🍭','🍬','🍫','🍿','🍩','🍪','🌰','🥜','🍯','🥛','🍼','🫖','☕️','🍵','🧃','🥤','🧋','🫙','🍶','🍺','🍻','🥂','🍷','🫗','🥃','🍸','🍹','🧉','🍾','🧊','🥄','🍴','🍽','🥣','🥡','🥢','🧂'
    ,'⚽️','🏀','🏈','⚾️','🥎','🎾','🏐','🏉','🥏','🎱','🪀','🏓','🏸','🏒','🏑','🥍','🏏','🪃','🥅','⛳️','🪁','🏹','🎣','🤿','🥊','🥋','🎽','🛹','🛼','🛷','⛸','🥌','🎿','⛷','🏂','🪂','🏋️‍♀️','🏋️','🏋️‍♂️','🤼‍♀️','🤼','🤼‍♂️','🤸‍♀️','🤸','🤸‍♂️','⛹️‍♀️','⛹️','⛹️‍♂️','🤺','🤾‍♀️','🤾','🤾‍♂️','🏌️‍♀️','🏌️','🏌️‍♂️','🏇','🧘‍♀️','🧘','🧘‍♂️','🏄‍♀️','🏄','🏄‍♂️','🏊‍♀️','🏊','🏊‍♂️','🤽‍♀️','🤽','🤽‍♂️','🚣‍♀️','🚣','🚣‍♂️','🧗‍♀️','🧗','🧗‍♂️','🚵‍♀️','🚵','🚵‍♂️','🚴‍♀️','🚴','🚴‍♂️','🏆','🥇','🥈','🥉','🏅','🎖','🏵','🎗','🎫','🎟','🎪','🤹','🤹‍♂️','🤹‍♀️','🎭','🩰','🎨','🎬','🎤','🎧','🎼','🎹','🥁','🪘','🪇','🎷','🎺','🪗','🎸','🪕','🎻','🪈','🎲','♟','🎯','🎳','🎮','🎰','🧩'
    ,'🚗','🚕','🚙','🚌','🚎','🏎','🚓','🚑','🚒','🚐','🛻','🚚','🚛','🚜','🦯','🦽','🦼','🛴','🚲','🛵','🏍','🛺','🚨','🚔','🚍','🚘','🚖','🛞','🚡','🚠','🚟','🚃','🚋','🚞','🚝','🚄','🚅','🚈','🚂','🚆','🚇','🚊','🚉','✈️','🛫','🛬','🛩','💺','🛰','🚀','🛸','🚁','🛶','⛵️','🚤','🛥','🛳','⛴','🚢','⚓️','🛟','🪝','⛽️','🚧','🚦','🚥','🚏','🗺','🗿','🗽','🗼','🏰','🏯','🏟','🎡','🎢','🛝','🎠','⛲️','⛱','🏖','🏝','🏜','🌋','⛰','🏔','🗻','🏕','⛺️','🛖','🏠','🏡','🏘','🏚','🏗','🏭','🏢','🏬','🏣','🏤','🏥','🏦','🏨','🏪','🏫','🏩','💒','🏛','⛪️','🕌','🕍','🛕','🕋','⛩','🛤','🛣','🗾','🎑','🏞','🌅','🌄','🌠','🎇','🎆','🌇','🌆','🏙','🌃','🌌','🌉','🌁'
    ,'⌚️','📱','📲','💻','⌨️','🖥','🖨','🖱','🖲','🕹','🗜','💽','💾','💿','📀','📼','📷','📸','📹','🎥','📽','🎞','📞','☎️','📟','📠','📺','📻','🎙','🎚','🎛','🧭','⏱','⏲','⏰','🕰','⌛️','⏳','📡','🔋','🪫','🔌','💡','🔦','🕯','🪔','🧯','🛢','🛍️','💸','💵','💴','💶','💷','🪙','💰','💳','💎','⚖️','🪮','🪜','🧰','🪛','🔧','🔨','⚒','🛠','⛏','🪚','🔩','⚙️','🪤','🧱','⛓','🧲','🔫','💣','🧨','🪓','🔪','🗡','⚔️','🛡','🚬','⚰️','🪦','⚱️','🏺','🔮','📿','🧿','🪬','💈','⚗️','🔭','🔬','🕳','🩹','🩺','🩻','🩼','💊','💉','🩸','🧬','🦠','🧫','🧪','🌡','🧹','🪠','🧺','🧻','🚽','🚰','🚿','🛁','🛀','🧼','🪥','🪒','🧽','🪣','🧴','🛎','🔑','🗝','🚪','🪑','🛋','🛏','🛌','🧸','🪆','🖼','🪞','🪟','🛍','🛒','🎁','🎈','🎏','🎀','🪄','🪅','🎊','🎉','🪩','🎎','🏮','🎐','🧧','✉️','📩','📨','📧','💌','📥','📤','📦','🏷','🪧','📪','📫','📬','📭','📮','📯','📜','📃','📄','📑','🧾','📊','📈','📉','🗒','🗓','📆','📅','🗑','🪪','📇','🗃','🗳','🗄','📋','📁','📂','🗂','🗞','📰','📓','📔','📒','📕','📗','📘','📙','📚','📖','🔖','🧷','🔗','📎','🖇','📐','📏','🧮','📌','📍','✂️','🖊','🖋','✒️','🖌','🖍','📝','✏️','🔍','🔎','🔏','🔐','🔒','🔓'
    ,'❤️','🩷','🧡','💛','💚','💙','🩵','💜','🖤','🩶','🤍','🤎','❤️‍🔥','❤️‍🩹','💔','❣️','💕','💞','💓','💗','💖','💘','💝','💟','☮️','✝️','☪️','🪯','🕉','☸️','✡️','🔯','🕎','☯️','☦️','🛐','⛎','♈️','♉️','♊️','♋️','♌️','♍️','♎️','♏️','♐️','♑️','♒️','♓️','🆔','⚛️','🉑','☢️','☣️','📴','📳','🈶','🈚️','🈸','🈺','🈷️','✴️','🆚','💮','🉐','㊙️','㊗️','🈴','🈵','🈹','🈲','🅰️','🅱️','🆎','🆑','🅾️','🆘','❌','⭕️','🛑','⛔️','📛','🚫','💯','💢','♨️','🚷','🚯','🚳','🚱','🔞','📵','🚭','❗️','❕','❓','❔','‼️','⁉️','🔅','🔆','〽️','⚠️','🚸','🔱','⚜️','🔰','♻️','✅','🈯️','💹','❇️','✳️','❎','🌐','💠','Ⓜ️','🌀','💤','🏧','🚾','♿️','🅿️','🛗','🈳','🈂️','🛂','🛃','🛄','🛅','🚹','🚺','🚼','⚧','🚻','🚮','🎦','🛜','📶','🈁','🔣','ℹ️','🔤','🔡','🔠','🆖','🆗','🆙','🆒','🆕','🆓','0️⃣','1️⃣','2️⃣','3️⃣','4️⃣','5️⃣','6️⃣','7️⃣','8️⃣','9️⃣','🔟','🔢','#️⃣','*️⃣','⏏️','▶️','⏸','⏯','⏹','⏺','⏭','⏮','⏩','⏪','⏫','⏬','◀️','🔼','🔽','➡️','⬅️','⬆️','⬇️','↗️','↘️','↙️','↖️','↕️','↔️','↪️','↩️','⤴️','⤵️','🔀','🔁','🔂','🔄','🔃','🎵','🎶','➕','➖','➗','✖️','🟰','♾','💲','💱','™️','©️','®️','〰️','➰','➿','🔚','🔙','🔛','🔝','🔜','✔️','☑️','🔘','🔴','🟠','🟡','🟢','🔵','🟣','⚫️','⚪️','🟤','🔺','🔻','🔸','🔹','🔶','🔷','🔳','🔲','▪️','▫️','◾️','◽️','◼️','◻️','🟥','🟧','🟨','🟩','🟦','🟪','⬛️','⬜️','🟫','🔈','🔇','🔉','🔊','🔔','🔕','📣','📢','👁‍🗨','💬','💭','🗯','♠️','♣️','♥️','♦️','🃏','🎴','🀄️','🕐','🕑','🕒','🕓','🕔','🕕','🕖','🕗','🕘','🕙','🕚','🕛','🕜','🕝','🕞','🕟','🕠','🕡','🕢','🕣','🕤','🕥','🕦','🕧'
    ,'🏳️','🏴','🏁','🚩','🏳️‍🌈','🏳️‍⚧️','🏴‍☠️','🇦🇫','🇦🇽','🇦🇱','🇩🇿','🇦🇸','🇦🇩','🇦🇴','🇦🇮','🇦🇶','🇦🇬','🇦🇷','🇦🇲','🇦🇼','🇦🇺','🇦🇹','🇦🇿','🇧🇸','🇧🇭','🇧🇩','🇧🇧','🇧🇾','🇧🇪','🇧🇿','🇧🇯','🇧🇲','🇧🇹','🇧🇴','🇧🇦','🇧🇼','🇧🇷','🇮🇴','🇻🇬','🇧🇳','🇧🇬','🇧🇫','🇧🇮','🇰🇭','🇨🇲','🇨🇦','🇮🇨','🇨🇻','🇧🇶','🇰🇾','🇨🇫','🇹🇩','🇨🇱','🇨🇳','🇨🇽','🇨🇨','🇨🇴','🇰🇲','🇨🇬','🇨🇩','🇨🇰','🇨🇷','🇨🇮','🇭🇷','🇨🇺','🇨🇼','🇨🇾','🇨🇿','🇩🇰','🇩🇯','🇩🇲','🇩🇴','🇪🇨','🇪🇬','🇸🇻','🇬🇶','🇪🇷','🇪🇪','🇪🇹','🇪🇺','🇫🇰','🇫🇴','🇫🇯','🇫🇮','🇫🇷','🇬🇫','🇵🇫','🇹🇫','🇬🇦','🇬🇲','🇬🇪','🇩🇪','🇬🇭','🇬🇮','🇬🇷','🇬🇱','🇬🇩','🇬🇵','🇬🇺','🇬🇹','🇬🇬','🇬🇳','🇬🇼','🇬🇾','🇭🇹','🇭🇳','🇭🇰','🇭🇺','🇮🇸','🇮🇳','🇮🇩','🇮🇷','🇮🇶','🇮🇪','🇮🇲','🇮🇱','🇮🇹','🇯🇲','🇯🇵','🎌','🇯🇪','🇯🇴','🇰🇿','🇰🇪','🇰🇮','🇽🇰','🇰🇼','🇰🇬','🇱🇦','🇱🇻','🇱🇧','🇱🇸','🇱🇷','🇱🇾','🇱🇮','🇱🇹','🇱🇺','🇲🇴','🇲🇰','🇲🇬','🇲🇼','🇲🇾','🇲🇻','🇲🇱','🇲🇹','🇲🇭','🇲🇶','🇲🇷','🇲🇺','🇾🇹','🇲🇽','🇫🇲','🇲🇩','🇲🇨','🇲🇳','🇲🇪','🇲🇸','🇲🇦','🇲🇿','🇲🇲','🇳🇦','🇳🇷','🇳🇵','🇳🇱','🇳🇨','🇳🇿','🇳🇮','🇳🇪','🇳🇬','🇳🇺','🇳🇫','🇰🇵','🇲🇵','🇳🇴','🇴🇲','🇵🇰','🇵🇼','🇵🇸','🇵🇦','🇵🇬','🇵🇾','🇵🇪','🇵🇭','🇵🇳','🇵🇱','🇵🇹','🇵🇷','🇶🇦','🇷🇪','🇷🇴','🇷🇺','🇷🇼','🇼🇸','🇸🇲','🇸🇦','🇸🇳','🇷🇸','🇸🇨','🇸🇱','🇸🇬','🇸🇽','🇸🇰','🇸🇮','🇬🇸','🇸🇧','🇸🇴','🇿🇦','🇰🇷','🇸🇸','🇪🇸','🇱🇰','🇧🇱','🇸🇭','🇰🇳','🇱🇨','🇵🇲','🇻🇨','🇸🇩','🇸🇷','🇸🇿','🇸🇪','🇨🇭','🇸🇾','🇹🇼','🇹🇯','🇹🇿','🇹🇭','🇹🇱','🇹🇬','🇹🇰','🇹🇴','🇹🇹','🇹🇳','🇹🇷','🇹🇲','🇹🇨','🇹🇻','🇻🇮','🇺🇬','🇺🇦','🇦🇪','🇬🇧','🏴󠁧󠁢󠁥󠁮󠁧󠁿','🏴󠁧󠁢󠁳󠁣󠁴󠁿','🏴󠁧󠁢󠁷󠁬󠁳󠁿','🇺🇳','🇺🇸','🇺🇾','🇺🇿','🇻🇺','🇻🇦','🇻🇪','🇻🇳','🇼🇫','🇪🇭','🇾🇪','🇿🇲','🇿🇼'
    ,'🫨','🩷','🩵','🩶','🫸','🫸🏻','🫸🏼','🫸🏽','🫸🏾','🫸🏿','🫷','🫷🏻','🫷🏼','🫷🏽','🫷🏾','🫷🏿','🫏','🫎','🪿','🐦‍⬛','🪽','🪼','🪻','🫛','🫚','🪭','🪮','🪈','🪇','🪯','🛜'
    ,'🫠','🫢','🫣','🫡','🫥','🫤','🥹','🫱','🫱🏻','🫱🏼','🫱🏽','🫱🏾','🫱🏿','🫲','🫲🏻','🫲🏼','🫲🏽','🫲🏾','🫲🏿','🫳','🫳🏻','🫳🏼','🫳🏽','🫳🏾','🫳🏿','🫴','🫴🏻','🫴🏼','🫴🏽','🫴🏾','🫴🏿','🫰','🫰🏻','🫰🏼','🫰🏽','🫰🏾','🫰🏿','🫵','🫵🏻','🫵🏼','🫵🏽','🫵🏾','🫵🏿','🫶','🫶🏻','🫶🏼','🫶🏽','🫶🏾','🫶🏿','🤝🏻','🤝🏼','🤝🏽','🤝🏾','🤝🏿','🫱🏻‍🫲🏼','🫱🏻‍🫲🏽','🫱🏻‍🫲🏾','🫱🏻‍🫲🏿','🫱🏼‍🫲🏻','🫱🏼‍🫲🏽','🫱🏼‍🫲🏾','🫱🏼‍🫲🏿','🫱🏽‍🫲🏻','🫱🏽‍🫲🏼','🫱🏽‍🫲🏾','🫱🏽‍🫲🏿','🫱🏾‍🫲🏻','🫱🏾‍🫲🏼','🫱🏾‍🫲🏽','🫱🏾‍🫲🏿','🫱🏿‍🫲🏻','🫱🏿‍🫲🏼','🫱🏿‍🫲🏽','🫱🏿‍🫲🏾','🫦','🫅','🫅🏻','🫅🏼','🫅🏽','🫅🏾','🫅🏿','🫃','🫃🏻','🫃🏼','🫃🏽','🫃🏾','🫃🏿','🫄','🫄🏻','🫄🏼','🫄🏽','🫄🏾','🫄🏿','🧌','🪸','🪷','🪹','🪺','🫘','🫗','🫙','🛝','🛞','🛟','🪬','🪩','🪫','🩼','🩻','🫧','🪪','🟰'
    ,'😮‍💨','😵‍💫','😶‍🌫️','❤️‍🔥','❤️‍🩹','🧔‍♀️','🧔🏻‍♀️','🧔🏼‍♀️','🧔🏽‍♀️','🧔🏾‍♀️','🧔🏿‍♀️','🧔‍♂️','🧔🏻‍♂️','🧔🏼‍♂️','🧔🏽‍♂️','🧔🏾‍♂️','🧔🏿‍♂️','💑🏻','💑🏼','💑🏽','💑🏾','💑🏿','💏🏻','💏🏼','💏🏽','💏🏾','💏🏿','👨🏻‍❤️‍👨🏻','👨🏻‍❤️‍👨🏼','👨🏻‍❤️‍👨🏽','👨🏻‍❤️‍👨🏾','👨🏻‍❤️‍👨🏿','👨🏼‍❤️‍👨🏻','👨🏼‍❤️‍👨🏼','👨🏼‍❤️‍👨🏽','👨🏼‍❤️‍👨🏾','👨🏼‍❤️‍👨🏿','👨🏽‍❤️‍👨🏻','👨🏽‍❤️‍👨🏼','👨🏽‍❤️‍👨🏽','👨🏽‍❤️‍👨🏾','👨🏽‍❤️‍👨🏿','👨🏾‍❤️‍👨🏻','👨🏾‍❤️‍👨🏼','👨🏾‍❤️‍👨🏽','👨🏾‍❤️‍👨🏾','👨🏾‍❤️‍👨🏿','👨🏿‍❤️‍👨🏻','👨🏿‍❤️‍👨🏼','👨🏿‍❤️‍👨🏽','👨🏿‍❤️‍👨🏾','👨🏿‍❤️‍👨🏿','👩🏻‍❤️‍👨🏻','👩🏻‍❤️‍👨🏼','👩🏻‍❤️‍👨🏽','👩🏻‍❤️‍👨🏾','👩🏻‍❤️‍👨🏿','👩🏻‍❤️‍👩🏻','👩🏻‍❤️‍👩🏼','👩🏻‍❤️‍👩🏽','👩🏻‍❤️‍👩🏾','👩🏻‍❤️‍👩🏿','👩🏼‍❤️‍👨🏻','👩🏼‍❤️‍👨🏼','👩🏼‍❤️‍👨🏽','👩🏼‍❤️‍👨🏾','👩🏼‍❤️‍👨🏿','👩🏼‍❤️‍👩🏻','👩🏼‍❤️‍👩🏼','👩🏼‍❤️‍👩🏽','👩🏼‍❤️‍👩🏾','👩🏼‍❤️‍👩🏿','👩🏽‍❤️‍👨🏻','👩🏽‍❤️‍👨🏼','👩🏽‍❤️‍👨🏽','👩🏽‍❤️‍👨🏾','👩🏽‍❤️‍👨🏿','👩🏽‍❤️‍👩🏻','👩🏽‍❤️‍👩🏼','👩🏽‍❤️‍👩🏽','👩🏽‍❤️‍👩🏾','👩🏽‍❤️‍👩🏿','👩🏾‍❤️‍👨🏻','👩🏾‍❤️‍👨🏼','👩🏾‍❤️‍👨🏽','👩🏾‍❤️‍👨🏾','👩🏾‍❤️‍👨🏿','👩🏾‍❤️‍👩🏻','👩🏾‍❤️‍👩🏼','👩🏾‍❤️‍👩🏽','👩🏾‍❤️‍👩🏾','👩🏾‍❤️‍👩🏿','👩🏿‍❤️‍👨🏻','👩🏿‍❤️‍👨🏼','👩🏿‍❤️‍👨🏽','👩🏿‍❤️‍👨🏾','👩🏿‍❤️‍👨🏿','👩🏿‍❤️‍👩🏻','👩🏿‍❤️‍👩🏼','👩🏿‍❤️‍👩🏽','👩🏿‍❤️‍👩🏾','👩🏿‍❤️‍👩🏿','🧑🏻‍❤️‍🧑🏼','🧑🏻‍❤️‍🧑🏽','🧑🏻‍❤️‍🧑🏾','🧑🏻‍❤️‍🧑🏿','🧑🏼‍❤️‍🧑🏻','🧑🏼‍❤️‍🧑🏽','🧑🏼‍❤️‍🧑🏾','🧑🏼‍❤️‍🧑🏿','🧑🏽‍❤️‍🧑🏻','🧑🏽‍❤️‍🧑🏼','🧑🏽‍❤️‍🧑🏾','🧑🏽‍❤️‍🧑🏿','🧑🏾‍❤️‍🧑🏻','🧑🏾‍❤️‍🧑🏼','🧑🏾‍❤️‍🧑🏽','🧑🏾‍❤️‍🧑🏿','🧑🏿‍❤️‍🧑🏻','🧑🏿‍❤️‍🧑🏼','🧑🏿‍❤️‍🧑🏽','🧑🏿‍❤️‍🧑🏾','👨🏻‍❤️‍💋‍👨🏻','👨🏻‍❤️‍💋‍👨🏼','👨🏻‍❤️‍💋‍👨🏽','👨🏻‍❤️‍💋‍👨🏾','👨🏻‍❤️‍💋‍👨🏿','👨🏼‍❤️‍💋‍👨🏻','👨🏼‍❤️‍💋‍👨🏼','👨🏼‍❤️‍💋‍👨🏽','👨🏼‍❤️‍💋‍👨🏾','👨🏼‍❤️‍💋‍👨🏿','👨🏽‍❤️‍💋‍👨🏻','👨🏽‍❤️‍💋‍👨🏼','👨🏽‍❤️‍💋‍👨🏽','👨🏽‍❤️‍💋‍👨🏾','👨🏽‍❤️‍💋‍👨🏿','👨🏾‍❤️‍💋‍👨🏻','👨🏾‍❤️‍💋‍👨🏼','👨🏾‍❤️‍💋‍👨🏽','👨🏾‍❤️‍💋‍👨🏾','👨🏾‍❤️‍💋‍👨🏿','👨🏿‍❤️‍💋‍👨🏻','👨🏿‍❤️‍💋‍👨🏼','👨🏿‍❤️‍💋‍👨🏽','👨🏿‍❤️‍💋‍👨🏾','👨🏿‍❤️‍💋‍👨🏿','👩🏻‍❤️‍💋‍👨🏻','👩🏻‍❤️‍💋‍👨🏼','👩🏻‍❤️‍💋‍👨🏽','👩🏻‍❤️‍💋‍👨🏾','👩🏻‍❤️‍💋‍👨🏿','👩🏻‍❤️‍💋‍👩🏻','👩🏻‍❤️‍💋‍👩🏼','👩🏻‍❤️‍💋‍👩🏽','👩🏻‍❤️‍💋‍👩🏾','👩🏻‍❤️‍💋‍👩🏿','👩🏼‍❤️‍💋‍👨🏻','👩🏼‍❤️‍💋‍👨🏼','👩🏼‍❤️‍💋‍👨🏽','👩🏼‍❤️‍💋‍👨🏾','👩🏼‍❤️‍💋‍👨🏿','👩🏼‍❤️‍💋‍👩🏻','👩🏼‍❤️‍💋‍👩🏼','👩🏼‍❤️‍💋‍👩🏽','👩🏼‍❤️‍💋‍👩🏾','👩🏼‍❤️‍💋‍👩🏿','👩🏽‍❤️‍💋‍👨🏻','👩🏽‍❤️‍💋‍👨🏼','👩🏽‍❤️‍💋‍👨🏽','👩🏽‍❤️‍💋‍👨🏾','👩🏽‍❤️‍💋‍👨🏿','👩🏽‍❤️‍💋‍👩🏻','👩🏽‍❤️‍💋‍👩🏼','👩🏽‍❤️‍💋‍👩🏽','👩🏽‍❤️‍💋‍👩🏾','👩🏽‍❤️‍💋‍👩🏿','👩🏾‍❤️‍💋‍👨🏻','👩🏾‍❤️‍💋‍👨🏼','👩🏾‍❤️‍💋‍👨🏽','👩🏾‍❤️‍💋‍👨🏾','👩🏾‍❤️‍💋‍👨🏿','👩🏾‍❤️‍💋‍👩🏻','👩🏾‍❤️‍💋‍👩🏼','👩🏾‍❤️‍💋‍👩🏽','👩🏾‍❤️‍💋‍👩🏾','👩🏾‍❤️‍💋‍👩🏿','👩🏿‍❤️‍💋‍👨🏻','👩🏿‍❤️‍💋‍👨🏼','👩🏿‍❤️‍💋‍👨🏽','👩🏿‍❤️‍💋‍👨🏾','👩🏿‍❤️‍💋‍👨🏿','👩🏿‍❤️‍💋‍👩🏻','👩🏿‍❤️‍💋‍👩🏼','👩🏿‍❤️‍💋‍👩🏽','👩🏿‍❤️‍💋‍👩🏾','👩🏿‍❤️‍💋‍👩🏿','🧑🏻‍❤️‍💋‍🧑🏼','🧑🏻‍❤️‍💋‍🧑🏽','🧑🏻‍❤️‍💋‍🧑🏾','🧑🏻‍❤️‍💋‍🧑🏿','🧑🏼‍❤️‍💋‍🧑🏻','🧑🏼‍❤️‍💋‍🧑🏽','🧑🏼‍❤️‍💋‍🧑🏾','🧑🏼‍❤️‍💋‍🧑🏿','🧑🏽‍❤️‍💋‍🧑🏻','🧑🏽‍❤️‍💋‍🧑🏼','🧑🏽‍❤️‍💋‍🧑🏾','🧑🏽‍❤️‍💋‍🧑🏿','🧑🏾‍❤️‍💋‍🧑🏻','🧑🏾‍❤️‍💋‍🧑🏼','🧑🏾‍❤️‍💋‍🧑🏽','🧑🏾‍❤️‍💋‍🧑🏿','🧑🏿‍❤️‍💋‍🧑🏻','🧑🏿‍❤️‍💋‍🧑🏼','🧑🏿‍❤️‍💋‍🧑🏽','🧑🏿‍❤️‍💋‍🧑🏾'
]

def random_emoji_string(length):
    return ''.join(random.choices(emojis, k=length))

# write wildcard concerts to ical
c = Calendar()
for index, row in df[df['m_rating'] == '🤡Wildcard'].iterrows():
    e = Event()

    # add wildcard-name! Who needs text when you're gonna watch a wildcard concert!!
    e.name = "🤡: " + row['emoji_beskrivelse']
    e.location = row['location']

    e.begin = row['start_datetime']

    # assume all concerts are 1 hour
    e.end = row['start_datetime'] + timedelta(hours=1) # assume all concerts are 1 hour
    
    e.description = random_emoji_string(25)


    c.events.add(e)

# c.events
with open('wildcard.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())
